<a href="https://colab.research.google.com/github/DavidTorres-sys/proyecto-modelos-y-simulacion-udea/blob/main/03%20-%20SMOTE%20y%20XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports Necesarios

In [14]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import classification_report

from imblearn.over_sampling import SMOTE

from xgboost import XGBClassifier


In [15]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

In [16]:
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.04GB/s]


In [17]:
!unzip udea*.zip > /dev/null
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total


In [18]:
def read_file(file):
    try:
        return pd.read_csv(file)
    except FileNotFoundError:
        print(f"Error: El archivo '{file}' no se encontró.")
    except Exception as e:
        print(f"Ocurrió un error al leer el archivo: {e}")

In [19]:
df = read_file("train.csv")
print("Tamaño del dataset:", df.shape)
df.head()

Tamaño del dataset: (692500, 21)


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


# 1. Cargamos los datos

In [20]:
# Cargamos datasets de entrenamiento y prueba
df = read_file("train.csv")
test_df = read_file("test.csv")

# Inspección inicial
print("Train shape:", df.shape)
print("Test shape:", test_df.shape)
df.head()

Train shape: (692500, 21)
Test shape: (296786, 20)


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294



# 2. CLEANING + NUMERIC/CATEGORICAL DETECTION

In [21]:
# Función para separar columnas numéricas y categóricas
def detect_numeric(df):
    num_cols, cat_cols = [], []
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col], errors="raise")
            num_cols.append(col)
        except:
            cat_cols.append(col)
    return num_cols, cat_cols

train_num_cols, train_cat_cols = detect_numeric(df)
test_num_cols, test_cat_cols = detect_numeric(test_df)

# Relleno de valores faltantes
df[train_num_cols] = df[train_num_cols].fillna(df[train_num_cols].median())
df[train_cat_cols] = df[train_cat_cols].fillna("DESCONOCIDO")

test_df[test_num_cols] = test_df[test_num_cols].fillna(test_df[test_num_cols].median())
test_df[test_cat_cols] = test_df[test_cat_cols].fillna("DESCONOCIDO")

# Eliminar columnas duplicadas
df = df.loc[:, ~df.columns.duplicated()]
test_df = test_df.loc[:, ~test_df.columns.duplicated()]


# 3. TARGET DEFINITION

In [22]:
# Verificamos si existe la columna 'RENDIMIENTO_GLOBAL'
if "RENDIMIENTO_GLOBAL" in df.columns:
    y = df["RENDIMIENTO_GLOBAL"]
else:
    df["DESEMPENO"] = df[["INDICADOR_1","INDICADOR_2","INDICADOR_3","INDICADOR_4"]].mean(axis=1)
    df["RENDIMIENTO_GLOBAL"] = pd.qcut(
        df["DESEMPENO"], q=4,
        labels=["bajo","medio-bajo","medio-alto","alto"]
    )
    y = df["RENDIMIENTO_GLOBAL"]

X = df.drop(columns=["RENDIMIENTO_GLOBAL","ID"], errors="ignore")


# 4. ENCODING — OrdinalEncoder

In [23]:
# Detectamos columnas categóricas
cat_cols = X.select_dtypes(include=["object"]).columns
oe = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
X[cat_cols] = oe.fit_transform(X[cat_cols])

# Preparamos test_features
test_features = test_df.drop(columns=["ID"], errors="ignore")
for col in cat_cols:
    if col not in test_features.columns:
        test_features[col] = "DESCONOCIDO"

test_features = test_features[X.columns]
test_features[cat_cols] = oe.transform(test_features[cat_cols])

# 5. TRAIN/VALIDATION SPLIT

In [24]:
class_mapping = {"bajo": 0, "medio-bajo": 1, "medio-alto": 2, "alto": 3}
y_num = y.map(class_mapping)
# Dividimos los datos en entrenamiento y validación
# Estratificamos para mantener proporción de clases
X_train, X_val, y_train, y_val = train_test_split(
    X, y_num, test_size=0.25, random_state=42, stratify=y_num
)

# 6. SMOTE (balanceo de clases)

In [25]:
# Aplicamos SMOTE para generar más ejemplos de clases minoritarias
sm = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = sm.fit_resample(X_train, y_train)


# 7. XGBOOST MODEL

In [26]:
# Inicializamos modelo XGBoost para clasificación multi-clase
model = XGBClassifier(
    n_estimators=600,
    learning_rate=0.03,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="multi:softmax",
    num_class=4,
    eval_metric="mlogloss",
    random_state=42,
    use_label_encoder=False
)

model.fit(X_train_resampled, y_train_resampled)

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [16:49:39] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.03, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=600, n_jobs=None, num_class=4, ...)

# 8. VALIDATION

In [27]:
pred_val = model.predict(X_val)
print("===== VALIDATION REPORT =====")
print(classification_report(y_val, pred_val, target_names=["bajo","medio-bajo","medio-alto","alto"]))

===== VALIDATION REPORT =====
              precision    recall  f1-score   support

        bajo       0.46      0.56      0.50     43247
  medio-bajo       0.33      0.27      0.30     43068
  medio-alto       0.33      0.26      0.29     42905
        alto       0.54      0.63      0.58     43905

    accuracy                           0.43    173125
   macro avg       0.42      0.43      0.42    173125
weighted avg       0.42      0.43      0.42    173125



# 9. PREDICT TEST SET

In [28]:
test_predictions_num = model.predict(test_features)

# Mapear de vuelta a etiquetas originales
inv_class_mapping = {v: k for k, v in class_mapping.items()}
test_predictions = [inv_class_mapping[i] for i in test_predictions_num]


# 10. BUILD SUBMISSION CSV


In [29]:
# Construimos el DataFrame final para Kaggle
submission = pd.DataFrame({
    "ID": test_df["ID"],
    "RENDIMIENTO_GLOBAL": test_predictions
})

# Guardamos CSV listo para enviar
submission.to_csv("submission_xgb.csv", index=False)
print("\nSUCCESS: submission_xgb.csv created!")



SUCCESS: submission_xgb.csv created!


In [31]:
!kaggle competitions submit -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia -f submission_xgb.csv -m "raul ramos submission with linear model"

100% 4.04M/4.04M [00:00<00:00, 6.46MB/s]
400 Client Error: Bad Request for url: https://www.kaggle.com/api/v1/competitions/submissions/submit/udea-ai-4-eng-20252-pruebas-saber-pro-colombia
